# Font recognition - improved models


## Load data and train-validation split

**Data is loaded, observed and treated**

In [1]:
import numpy as np
import pandas as pd

train_data = pd.read_csv('data/train_data.csv')
train_labels = pd.read_csv('data/train_labels.csv')


Labels are factorized and a full dataframe is constructed adding the encoded values as the last column

In [2]:
label_encoded, unique_labels = pd.factorize(train_labels['Font'])
labels = pd.DataFrame(label_encoded, columns=['label'])
df = pd.concat([train_data, labels], axis = 1)

**Train and validation split is conducted**

In [3]:
from sklearn.model_selection import train_test_split
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]
x_train_df, x_valid_df, y_train_df, y_valid_df = train_test_split(X, Y, test_size=0.3, random_state = 0)

**Finally, the test data is loaded as well**

In [4]:
test_data = pd.read_csv('data/test_data.csv')
x_test_df = test_data

## Normalization of data

Now df has all the needed information. It will be transformed to a np.array for easier treatment within sklearn package

In [5]:
x_train_pre_norm = np.array(x_train_df)
x_valid_pre_norm = np.array(x_valid_df)
y_train = np.array(y_train_df)
y_valid = np.array(y_valid_df)
x_test_pre_norm = np.array(x_test_df)

X_np = np.array(X)

`mean` and `std` are obtained from full dataset

In [6]:
mean = np.sum(X_np, axis = 0) / X_np.shape[0]
std = np.std(X_np, axis = 0)

Implement normalization function from Homework 9

In [7]:
def normalize(X, mean, std):
    """Normalizes a given array X by columns 
    with the mean and std"""
    X_out = np.zeros(X.shape)
    X_out = (X - mean)/std
    return X_out 

In [8]:
x_train = normalize(x_train_pre_norm, mean, std)
x_valid = normalize(x_valid_pre_norm, mean, std)
x_test = normalize(x_test_pre_norm, mean, std)

## Function to save submission csv

A function will be created that saves predictions as a csv with the correct format

In [9]:
def predictions_as_csv(y_pred, file_name):
    path = "submissions/"
    status = 0
    if len(y_pred) == 29221:
        ids = np.arange(1,len(y_pred)+1,1)
        pred_label = unique_labels[y_pred]
        data = {'ID':ids, 'Font':pred_label} 
        submission = pd.DataFrame(data)
        submission.to_csv(path + file_name + ".csv", index = False)
        status = 1
    
    return status

## Neural network

In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import hamming_loss

In [11]:
model_nNetwork = MLPClassifier(random_state=1, max_iter=300)
model_nNetwork.fit(x_train, y_train)

C:\Users\norio\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:587: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(max_iter=300, random_state=1)

In [12]:
y_pred_train = model_nNetwork.predict(x_train)
error = hamming_loss(y_train, y_pred_train)
print('The training error is: ' + str(error) + '.')

The training error is: 0.3168131868131868.


In [13]:
y_pred_valid = model_nNetwork.predict(x_valid)
error = hamming_loss(y_valid, y_pred_valid)
print('The validation error is: ' + str(error) + '.')

The validation error is: 0.39887179487179486.


In [14]:
predicted_score = 1- error
predicted_score

0.6011282051282052

**Predictions with test set are computed**

In [15]:
y_pred_test = model_nNetwork.predict(x_test)

In [16]:
predictions_as_csv(y_pred_test, "nNetwork_prediction")

1

## Neural network alpha = 0.5

In [17]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import hamming_loss

In [ ]:
model_nNetwork = MLPClassifier(random_state=1, max_iter=300, alpha = 0.5, activation = 'logistic')
model_nNetwork.fit(x_train, y_train)

In [ ]:
y_pred_train = model_nNetwork.predict(x_train)
error = hamming_loss(y_train, y_pred_train)
print('The training error is: ' + str(error) + '.')

In [ ]:
y_pred_valid = model_nNetwork.predict(x_valid)
error = hamming_loss(y_valid, y_pred_valid)
print('The validation error is: ' + str(error) + '.')

In [ ]:
predicted_score = 1- error
predicted_score

**Predictions with test set are computed**

In [ ]:
y_pred_test = model_nNetwork.predict(x_test)

In [ ]:
predictions_as_csv(y_pred_test, "nNetwork_prediction2")

## Neural network varying params

A variation of parameters is tried to see f performance can be increased. It takes too much time to try out all compbinations so several of them were tried and results are shown below.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import hamming_loss

In [ ]:
activation_functions = ['logistic', 'tanh', 'relu', 'identity']
solvers = ['adam']
learning_rates = ['adaptive']
alphas = np.linspace(start=1, stop =10, num = 10)

In [ ]:
rows_NN_performance = []
columns = ['activation_function', 'solver', 'learning_rate', 'alpha', 'train_error', 'test_error', 'predicted_score']


In [ ]:
for activation_function in activation_functions:
    print("\n=============")
    print("Activation function")
    print(activation_function)
    for solver in solvers:
        print("\n======")
        print("Solver")
        print(solver)
        
        for learning_rate in learning_rates:
            print("=")
            print("L rate")
            print(learning_rate)
            
            for alpha in alphas:
                print(alpha)
                model_nNetwork = MLPClassifier(random_state=1, max_iter=600, alpha = alpha, activation = activation_function, solver = solver)
                model_nNetwork.fit(x_train, y_train)
                
                y_pred_train = model_nNetwork.predict(x_train)
                train_error = hamming_loss(y_train, y_pred_train)
                
                y_pred_valid = model_nNetwork.predict(x_valid)
                test_error = hamming_loss(y_valid, y_pred_valid)
                
                predicted_score = 1- test_error
                predicted_score
                
                rows_NN_performance.append(dict(zip(columns,[activation_function, solver, learning_rate, alpha, train_error, test_error, predicted_score ])))                

df = pd.DataFrame(rows_NN_performance)                

In [ ]:
df

## Neural network more hidden layers

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import hamming_loss

In [ ]:
model_nNetwork = MLPClassifier(random_state=1, max_iter=500, alpha = 0.5, activation = 'logistic',  hidden_layer_sizes=(10, 6))
model_nNetwork.fit(x_train, y_train)

In [ ]:
y_pred_train = model_nNetwork.predict(x_train)
error = hamming_loss(y_train, y_pred_train)
print('The training error is: ' + str(error) + '.')

In [ ]:
y_pred_valid = model_nNetwork.predict(x_valid)
error = hamming_loss(y_valid, y_pred_valid)
print('The validation error is: ' + str(error) + '.')

In [ ]:
predicted_score = 1- error
predicted_score

## Random search

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import hamming_loss
from scipy.stats import uniform

In [ ]:
activation_functions = ['logistic', 'tanh', 'relu', 'identity']
solvers = ['adam','lbfgs', 'sgd']
learning_rates = ['adaptive', 'invscaling']
alphas = np.linspace(start=0.1, stop =3, num = 10).tolist()

In [ ]:
distributions = {'activation':('logistic', 'tanh', 'relu', 'identity'), 'solver':('adam','lbfgs', 'sgd'), 'learning_rate':('adaptive', 'invscaling'), 'alpha':alphas}

In [ ]:
neural_NN = MLPClassifier(max_iter=1000, random_state = 1)

In [ ]:
#clf = GridSearchCV(neural_NN, parameters)
clf = RandomizedSearchCV(neural_NN, distributions)

In [ ]:
clf.fit(x_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
y_pred_train = clf.predict(x_train)
error = hamming_loss(y_train, y_pred_train)
print('The training error is: ' + str(error) + '.')

In [ ]:
y_pred_valid = clf.predict(x_valid)
error = hamming_loss(y_valid, y_pred_valid)
print('The validation error is: ' + str(error) + '.')

In [ ]:
predicted_score = 1- error
predicted_score

In [ ]:
sorted(clf.cv_results_.keys())

## Deep neural network

In [ ]:
model_nNetwork = MLPClassifier(random_state=1, max_iter=1000, alpha = 1.38, activation = 'relu', learning_rate = 'adaptive', solver = 'sgd', hidden_layer_sizes=(200,100,100,50) )
model_nNetwork.fit(x_train, y_train)

y_pred_train = model_nNetwork.predict(x_train)
error = hamming_loss(y_train, y_pred_train)
print('The training error is: ' + str(error) + '.')

y_pred_valid = model_nNetwork.predict(x_valid)
error = hamming_loss(y_valid, y_pred_valid)
print('The validation error is: ' + str(error) + '.')

predicted_score = 1- error
predicted_score